# Pre-processing data for Filter view

## Grab all relevant data by story id from postgres
Replaced by next cell

In [43]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("connected!")

cur.execute('''
    SELECT *
    FROM
        (SELECT x_apyt.*,s.section_name
        FROM
            (SELECT x_apy.*,t.theme_name
            FROM
                (SELECT x_ap.*,y.stype_name
                FROM
                    (SELECT x_a.*, p.publisher_name
                    FROM
                        (SELECT story.*, a.author_name
                        FROM story
                        LEFT JOIN
                            (SELECT authorship.story_id, author.author_name
                            FROM authorship
                            LEFT JOIN author ON authorship.author_id=author.author_id) AS a
                        ON story.story_id = a.story_id) AS x_a
                    LEFT JOIN
                        (SELECT *
                        FROM publishing
                        LEFT JOIN publisher ON publishing.publisher_id=publisher.publisher_id) AS p
                    ON x_a.story_id=p.story_id) AS x_ap
                LEFT JOIN
                    (SELECT *
                    FROM styping
                    LEFT JOIN stype ON styping.stype_id=stype.stype_id) as y
                ON x_ap.story_id=y.story_id) AS x_apy
            LEFT JOIN
                (SELECT *
                FROM theming
                LEFT JOIN theme ON theming.theme_id=theme.theme_id) as t
            ON x_apy.story_id=t.story_id) AS x_apyt
        LEFT JOIN
            (SELECT *
            FROM sectioning
            LEFT JOIN section ON sectioning.section_id=section.section_id) as s
        ON x_apyt.story_id=s.story_id) AS x_apyts
    LEFT JOIN
        (SELECT *
        FROM sinstance
        LEFT JOIN user_gazetteer ON sinstance.place_id=user_gazetteer.place_id) as gaz
    ON x_apyts.story_id=gaz.story_id
    ORDER BY x_apyts.story_id ASC
    ''')
rawStories = cur.fetchall()
#print(rawStories)
print()
print("successfully loaded all stories in raw (repeated) format, ready for filtering")
print()

#close the connection
conn.close()
print("connection closed")

connected!

successfully loaded all stories in raw (repeated) format, ready for filtering

connection closed


## Prepare for return to reader filter results view

In [91]:
import pandas as pd
import geopandas as gpd
import geojson
import psycopg2
import os

os.remove(r"rawStories.geojson")

conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("connected!")

sql = '''
    SELECT *
    FROM
        (SELECT x_apyt.*,s.section_name
        FROM
            (SELECT x_apy.*,t.theme_name
            FROM
                (SELECT x_ap.*,y.stype_name
                FROM
                    (SELECT x_a.*, p.publisher_name
                    FROM
                        (SELECT story.*, a.author_name
                        FROM story
                        LEFT JOIN
                            (SELECT authorship.story_id, author.author_name
                            FROM authorship
                            LEFT JOIN author ON authorship.author_id=author.author_id) AS a
                        ON story.story_id = a.story_id) AS x_a
                    LEFT JOIN
                        (SELECT *
                        FROM publishing
                        LEFT JOIN publisher ON publishing.publisher_id=publisher.publisher_id) AS p
                    ON x_a.story_id=p.story_id) AS x_ap
                LEFT JOIN
                    (SELECT *
                    FROM styping
                    LEFT JOIN stype ON styping.stype_id=stype.stype_id) as y
                ON x_ap.story_id=y.story_id) AS x_apy
            LEFT JOIN
                (SELECT *
                FROM theming
                LEFT JOIN theme ON theming.theme_id=theme.theme_id) as t
            ON x_apy.story_id=t.story_id) AS x_apyt
        LEFT JOIN
            (SELECT *
            FROM sectioning
            LEFT JOIN section ON sectioning.section_id=section.section_id) as s
        ON x_apyt.story_id=s.story_id) AS x_apyts
    LEFT JOIN
        (SELECT *
        FROM sinstance
        LEFT JOIN user_gazetteer ON sinstance.place_id=user_gazetteer.place_id) as gaz
    ON x_apyts.story_id=gaz.story_id
    ORDER BY x_apyts.story_id ASC
    '''
rawStories = gpd.read_postgis(sql, con=conn, geom_col="geom", parse_dates = {"publish_date", "t_begin", "t_end"})
#rawStories = gpd.read_postgis(sql, con=conn, geom_col="geom")
print("Raw stories loaded")
print()
print(rawStories)
print()
print("Assigned datetime to geodataframe")
rawStories["publish_date"] = rawStories['publish_date'].dt.strftime('%x') #https://strftime.org/
rawStories["t_begin"] = rawStories['t_begin'].dt.strftime("%x")
rawStories["t_end"] = rawStories['t_end'].dt.strftime("%x")
#rawStories["publish_date"] = pd.to_datetime(rawStories['publish_date'])
#rawStories["t_begin"] = pd.to_datetime(rawStories['t_begin'])
#rawStories["t_end"] = pd.to_datetime(rawStories['t_end'])
print("Unassigned datetime to geopandas df, because GEOJSON doesn't like it")
print()
print(rawStories)
rawStories.info()
rawStories.to_file("rawStories.geojson", driver='GeoJSON')
#rawStories.to_json()


connected!
Raw stories loaded

                                               title  \
0  Esta cidade ja é para animais vadios: gatos de...   
1  A avó da Uber Eats em Alfama é a salvaçao dos ...   
2  Alvalade: o bairro que não é para turistas tem...   
3  "As ""Gaulesas"" que defendem a Mouraria da es...   
4  Mais floreiras e menos carros. Estes lisboetas...   
5  Nos Anjos, o ativismo passou a fazer-se ao mic...   
6  Quem é Alexis Lapas, o peixeiro de Benfica que...   
7  A guerra das bicicletas passou para a Avenida ...   

                                             summary  \
0  Vai longe o tempo em que Lisboa estava cheia d...   
1  A mercearia Castanheira, da Dona Manuela, não ...   
2  Todos os bairros de Lisboa sentem a crise do c...   
3  São três mulheres que ajudam os habitantes da ...   
4  Miguel Macedo e Silke Jellen, a vizinha de bai...   
5  A emitir a partir dos Anjos, esta rádio quer s...   
6  "Tem um nome real ainda mais estranho do que n...   
7  A questão das

# Add news entries to backend
https://geopandas.org/docs/reference/api/geopandas.GeoDataFrame.to_postgis.html